## GPT 2

In [1]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

c:\Users\DELL\anaconda3\envs\ai-backend\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I am in Sri Lanka,", max_length=30, num_return_sequences=1)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I am in Sri Lanka, but what I do in that country is different—I've been here for nearly 12 years, and I've"}]

In [ ]:
## FINE TUNE GPT2

In [6]:
!pip install transformers datasets

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     -------------------- ----------------- 30.7/57.7 kB 660.6 kB/s eta 0:00:01
     ---------------------------------------- 57.7/57.7 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/480.6 kB ? eta -:--:--
   -------- ------------------------------- 102.4/480.6 kB 2.0 MB/s eta 0:00:01
   ------------------- -------------------- 235.5/480.6 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 480.6/480.6 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/450.5 kB ? eta -:--:--
   ----------------- --------------------- 204.8/450.5 kB 12.2 MB/s eta 0:00:01
   ----------------- --------------------- 204.8/450.5 kB 12.2 MB/s eta 0:00:01
   ----------------- --------------------- 204.8/450.5 kB 12.2 MB/s eta 0:00:01
   ----------------- --------------------- 204.8/450.5 kB 12.2 MB/s eta 0:00:01


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.16.0 requires torch==2.1.0, but you have torch 2.5.1 which is incompatible.


In [15]:
!pip install --upgrade accelerate


In [17]:
!pip install torch



In [7]:
import os
import torch
from datasets import load_dataset

In [22]:
!pip install --upgrade accelerate transformers datasets torch

['Requirement already satisfied: accelerate in c:\\users\\dell\\anaconda3\\envs\\ai-backend\\lib\\site-packages (1.2.1)',
 'Requirement already satisfied: transformers in c:\\users\\dell\\anaconda3\\envs\\ai-backend\\lib\\site-packages (4.37.2)',
 'Collecting transformers',
 '  Downloading transformers-4.47.1-py3-none-any.whl.metadata (44 kB)',
 '     ---------------------------------------- 0.0/44.1 kB ? eta -:--:--',
 '     --------- ------------------------------ 10.2/44.1 kB ? eta -:--:--',
 '     ----------------- -------------------- 20.5/44.1 kB 165.2 kB/s eta 0:00:01',
 '     -------------------------- ----------- 30.7/44.1 kB 187.9 kB/s eta 0:00:01',
 '     -------------------------------------- 44.1/44.1 kB 216.9 kB/s eta 0:00:00',
 'Requirement already satisfied: datasets in c:\\users\\dell\\anaconda3\\envs\\ai-backend\\lib\\site-packages (3.2.0)',
 'Requirement already satisfied: torch in c:\\users\\dell\\anaconda3\\envs\\ai-backend\\lib\\site-packages (2.5.1)',
 'Requireme

In [39]:
# ========================================
# 1. Import Required Libraries
# ========================================

import torch
from torch.utils.data import DataLoader
from transformers import (
    GPT2Tokenizer, 
    GPT2LMHeadModel, 
    DataCollatorForLanguageModeling, 
    AdamW
)

In [40]:
from datasets import load_dataset
from tqdm import tqdm


In [41]:
#
# Step 1: Load & Prepare Data
#

# Example: load a text dataset from a file named 'train.txt'
# Each line is treated as a separate example
dataset = load_dataset("text", data_files={"train": "train.txt"})

# Filter out empty lines if your file has blank lines
def remove_empty_strings(example):
    return len(example["text"].strip()) > 0

dataset["train"] = dataset["train"].filter(remove_empty_strings)

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 has no real pad token

# A function to tokenize each example (one line)
def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        truncation=True, 
        max_length=128, 
        padding="max_length"
    )

# Apply the tokenizer
tokenized_dataset = dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"])

# Create a DataCollator for causal language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Turn the tokenized dataset into a PyTorch Dataset 
# (datasets.Dataset already works like a PyTorch dataset, but we can make a DataLoader)
train_dataset = tokenized_dataset

# Build a DataLoader
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=2, 
    shuffle=True, 
    collate_fn=data_collator
)

Map: 100%|██████████| 16/16 [00:00<00:00, 75.41 examples/s] 


In [42]:
#
# Step 2: Initialize Model & Optimizer
#

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Adjust embeddings if the tokenizer size changed
model.config.pad_token_id = model.config.eos_token_id  # Avoid warnings about missing pad_token

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Use AdamW optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)


c:\Users\DELL\anaconda3\envs\ai-backend\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  num_warmup_steps (`int`):


In [43]:
#
# Step 3: Training Loop
#

num_epochs = 3

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    total_loss = 0.0

    # Iterate over batches
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        # batch is a dict from the collator: {input_ids, attention_mask, labels, etc.}
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)  # for causal LM, usually same as input_ids

        # Forward pass
        outputs = model(input_ids=input_ids, 
                        attention_mask=attention_mask, 
                        labels=labels)
        loss = outputs.loss

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Average Loss: {avg_loss:.4f}")


Epoch 1/3: 100%|██████████| 8/8 [00:37<00:00,  4.74s/it]


Epoch 1 - Average Loss: 3.5970


Epoch 2/3: 100%|██████████| 8/8 [00:21<00:00,  2.71s/it]


Epoch 2 - Average Loss: 1.9058


Epoch 3/3: 100%|██████████| 8/8 [00:21<00:00,  2.65s/it]

Epoch 3 - Average Loss: 1.0643


In [45]:
#
# Step 4: Save the Fine-Tuned Model
#

model.save_pretrained("gpt2-finetuned-custom-loop")
tokenizer.save_pretrained("gpt2-finetuned-custom-loop")

print("Fine-tuning complete!")

Fine-tuning complete!


In [46]:
### Using the fine tuned model

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline

model_path = "gpt2-finetuned-custom-loop"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Caller: Hello, I have a question about my account.\nCallee: Yes, can you please provide your account number?"
outputs = generator(
    prompt,
    max_length=200,     # a larger max length
    num_return_sequences=1,
    do_sample=True,     # to allow more "creative" generation
    top_k=50,
    top_p=0.95
)
print(outputs[0]["generated_text"])

# print(outputs[0]["generated_text"])


c:\Users\DELL\anaconda3\envs\ai-backend\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\DELL\anaconda3\envs\ai-backend\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Caller: Hello, I have a question about my account.
Callee: Yes, can you please provide your account number? Can I provide it immediately? Can I immediately transfer it to your bank account?
’s Free Banking: Hello, I’’m calling from your bank directly. Can I provide a new banking number immediately?’s a number that I need to confirm immediately? Your details can be shared with our partners immediately. Can I provide a more comprehensive offer immediately?’s available immediately?’s up-to-date. Can you provide a credit card number immediately?’s up-to-date?’s available for free? Can I make an appointment now?’s a good idea? Can I call your bank directly?’s ready to discuss your new offer now?’s an urgent matter. Can I provide your details directly now?’s up to date?�
Caller: Hello, I have a question about my account.
Callee: Yes, can you please provide your account number? Can I provide it immediately? Can I immediately transfer it to your bank account?
’s Free Banking: Hello, I’’m calli

In [51]:
conversation_history = """Caller: Hello, I have a question about my account.\nCallee:"""
while True:
    # Generate the Callee's response
    outputs = generator(
        conversation_history,
        max_length=200,
        pad_token_id=tokenizer.eos_token_id,  # handle GPT-2's lack of pad token
        num_return_sequences=1,
        do_sample=True
    )
    generated_text = outputs[0]["generated_text"]

    # Extract only the newly generated part after 'Callee:'
    new_text = generated_text[len(conversation_history):]

    # Print or store the Callee's response
    print("Callee:", new_text.strip())

    # Then get user input (Caller) to continue the conversation
    user_input = input("Caller: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    # Append the new user line to conversation
    conversation_history += f"Callee:{new_text}\nCaller: {user_input}\nCallee:"

Callee: Could you provide more details regarding this matter? Can you send an email instead?
Callee: That sounds important. What kind of promotion is it? I’d like to make sure I received it before I signed it. Can I send it via the official website?
Callee: Sure, let me check your account immediately. What kind of promotion is it? What kind of promotion is it? What kind of promotion should I send it via?
Callee: I’d like to make sure I received it before I signed it. Can I send it via the official website?’s an automated messaging system that automatically sends you an automated email? Can I send it via the official website instead?’s not an automated message? Could you authorize it? Can I send it via the official website instead?’s not an automated message? Can
